### Load dependencies

In [1]:
import os
import xarray as xr
import numpy as np
import geopandas as gpd

# pyflwdir
import pyflwdir

# hydromt
from hydromt import DataCatalog, flw

# plot
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import cartopy.crs as ccrs

### Deriving flow directions from Elevation data

First let's read the data, and prepare the data to use in pyflowdir:

In [2]:
# Read data
local_dem = xr.open_dataset("../data/local_raw/xgeo_dem.nc")
local_dem.raster.set_crs(25833)

# To orient data from north to south (crucial for pyflwdir!)
da_elv = local_dem["xgeo_dem_2"].sortby("y", ascending=False)

# Set nodata
da_elv = da_elv.where(~da_elv.isnull(), -9999)
da_elv.raster.set_nodata(-9999)

To derive flow directions from a DEM, you can use the [hydromt.flw.d8_from_dem](https://deltares.github.io/hydromt/latest/_generated/hydromt.flw.d8_from_dem.html#hydromt.flw.d8_from_dem) method of HydroMT.

This method derives D8 flow directions grid from an elevation grid and allows several options to the users:
 - **outlets**: outlets can be defined at ``edge``s of the grid (defualt) or force all flow to go to the minimum elevation point ``min``. The latter only makes sense if your DEM only is masked to the catchment. Additionnally, the user can also force specific pits locations via ``idxs_pit``.
 - **depression filling**: local depressions are filled based on their lowest pour point level if the pour point depth is smaller than the maximum pour point depth ``max_depth``, otherwise the lowest elevation in the depression becomes a pit. By default ``max_depth`` is set to -1 m filling all local depressions.
- **river burning**: while it is already possible to provide a river vector layer ``gdf_stream`` with ``uparea`` (km2) column to further guide the derivation of flow directions, this option is currently being improved and not yet fully functional. See also HydroMT core PR [305](https://github.com/Deltares/hydromt/pull/305)

Let's see an example:

In [3]:
# Derive flow directions with outlets at the edges
local_dem["flwdir_derived"] = flw.d8_from_dem(
    da_elv=da_elv,
    gdf_stream=None,
    max_depth=-1,  # no local pits
    outlets="edge",
    idxs_pit=None,
)

## Deriving other DEM and flow directions related data

Once you are satisfied with your flow direction map, you can create additional derived variables like upstream area or streamorder that can prove useful for example to build a model based on ``subbasin`` region.

Here are some examples how to do that using PyFlwdir methods.

In [4]:
# Create a new adapted dataset with the riverburn flow directions
merit_adapted = da_elv.to_dataset(name="elevtn")
merit_adapted["flwdir"] = local_dem["flwdir_derived"]
dims = merit_adapted.raster.dims

# Create a PyFlwDir object from the dataset
flwdir = flw.flwdir_from_da(merit_adapted["flwdir"])

# uparea
uparea = flwdir.upstream_area(unit="km2")
merit_adapted["uparea"] = xr.Variable(dims, uparea, attrs=dict(_FillValue=-9999))

# stream order
strord = flwdir.stream_order()
merit_adapted["strord"] = xr.Variable(dims, strord)
merit_adapted["strord"].raster.set_nodata(255)

# slope
slope = pyflwdir.dem.slope(
    elevtn=merit_adapted["elevtn"].values,
    nodata=merit_adapted["elevtn"].raster.nodata,
    latlon=False,  # True if geographic crs, False if projected crs
    transform=merit_adapted["elevtn"].raster.transform,
)
merit_adapted["slope"] = xr.Variable(dims, slope)
merit_adapted["slope"].raster.set_nodata(merit_adapted["elevtn"].raster.nodata)

# basin at the pits locations
basins = flwdir.basins(idxs=flwdir.idxs_pit).astype(np.int32)
merit_adapted["basins"] = xr.Variable(dims, basins, attrs=dict(_FillValue=0))

# basin index file
gdf_basins = merit_adapted["basins"].raster.vectorize()

merit_adapted

<xarray.Dataset>
Dimensions:      (x: 1195, y: 1550)
Coordinates:
  * x            (x) float64 -7.5e+04 -7.4e+04 -7.3e+04 ... 1.118e+06 1.119e+06
  * y            (y) float64 7.999e+06 7.998e+06 ... 6.451e+06 6.45e+06
    lat          (y, x) float64 71.39 71.39 71.39 71.4 ... 57.76 57.76 57.76
    lon          (y, x) float64 -1.319 -1.292 -1.265 -1.238 ... 25.4 25.41 25.43
    spatial_ref  int32 0
Data variables:
    elevtn       (y, x) float64 -9.999e+03 -9.999e+03 ... -9.999e+03 -9.999e+03
    flwdir       (y, x) uint8 247 247 247 247 247 247 ... 247 247 247 247 247
    uparea       (y, x) float32 -9.999e+03 -9.999e+03 ... -9.999e+03 -9.999e+03
    strord       (y, x) uint8 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    slope        (y, x) float32 -9.999e+03 -9.999e+03 ... -9.999e+03 -9.999e+03
    basins       (y, x) int32 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0

### Exporting the newly created data and corresponding data catalog

Finally, once we are happy with the new dataset, we can write out the data and create the corresponding data catalog so that it can be re-used to build a new wflow model.

In [5]:
output_path = "../data/local_processed_demo"
filename = "local_dem"

# Exporting
# Export the gridded data as tif files in a new folder
# export the hydrography data as tif files (one per variable)
merit_adapted.raster.to_mapstack(
    root=os.path.join(output_path, filename),
    driver="GTiff",
)

# export the basin index as geosjon
gdf_basins.to_file(
    os.path.join(output_path, f"{filename}_basins.geojson"), driver="GeoJSON"
)

Now let's prepare the corresponding data catalog:

In [6]:
print(
    f"""
Add to data_catalog.yml
-----------------------
{filename}:
  data_type: RasterDataset
  driver: raster
  crs: 25833
  path: ./{filename}/{{variable}}.tif
  rename:
    slope: lndslp
  meta:
    category: topography
    processing_notes: prepared from xgeo_dem by deriving flow directions using pyflwdir.
    processing_script: preprocess_dem.py

{filename}_index:
  data_type: GeoDataFrame
  driver: vector
  crs: 25833
  path: ./{filename}_basins.geojson
  rename:
    value: basid
  meta:
    category: topography
    processing_notes: prepared from xgeo_dem by deriving flow directions using pyflwdir.
    processing_script: preprocess_dem.py

"""
)


Add to data_catalog.yml
-----------------------
local_dem:
  data_type: RasterDataset
  driver: raster
  crs: 25833
  path: ./local_dem/{variable}.tif
  rename:
    slope: lndslp
  meta:
    category: topography
    processing_notes: prepared from xgeo_dem by deriving flow directions using pyflwdir.
    processing_script: preprocess_dem.py

local_dem_index:
  data_type: GeoDataFrame
  driver: vector
  crs: 25833
  path: ./local_dem_basins.geojson
  rename:
    value: basid
  meta:
    category: topography
    processing_notes: prepared from xgeo_dem by deriving flow directions using pyflwdir.
    processing_script: preprocess_dem.py


